# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
#https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html

# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import re
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','stopwords'])
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer


from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /home/ajana/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ajana/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ajana/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/ajana/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('DisasterResponseTable',engine)
X = df.message.values
Y = df.iloc[:,4:]
X[1:10]

array(['Is the Hurricane over or is it not over',
       'Looking for someone but no name',
       'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.',
       'says: west side of Haiti, rest of the country today and tonight',
       'Information about the National Palace-',
       'Storm at sacred heart of jesus',
       'Please, we need tents and water. We are in Silo, Thank you!',
       'I would like to receive the messages, thank you',
       'I am in Croix-des-Bouquets. We have health issues. They ( workers ) are in Santo 15. ( an area in Croix-des-Bouquets )'],
      dtype=object)

### 2. Write a tokenization function to process your text data

In [4]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
stop_words = stopwords.words("english")
        
# def tokenize(text):
#     detected_urls = re.findall(url_regex, text)
#     for url in detected_urls:
#         text = text.replace(url, "urlplaceholder")

#     tokens = word_tokenize(text)
#     lemmatizer = WordNetLemmatizer()
#     # lemmatize andremove stop words
#     tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

#     return tokens

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


In [5]:
tokenize(X[0])

['weather',
 'update',
 '-',
 'a',
 'cold',
 'front',
 'from',
 'cuba',
 'that',
 'could',
 'pas',
 'over',
 'haiti']

In [6]:

def display_results(y_test, y_pred):
    #labels = np.unique(y_pred)
    #confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    #print("Labels:", labels)
    #print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 100)))
    ])

In [10]:
xgb_pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(XGBClassifier(objective='binary:logistic')))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:

X_train, X_test, y_train, y_test = train_test_split(X, Y)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
%%time

# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

# display results
display_results(y_test, y_pred)

Accuracy: related                   0.810051
request                   0.897034
offer                     0.994467
aid_related               0.771016
medical_help              0.919010
medical_products          0.951130
search_and_rescue         0.971723
security                  0.982941
military                  0.968649
water                     0.953588
food                      0.925158
shelter                   0.924543
clothing                  0.984017
money                     0.976026
missing_people            0.988167
refugees                  0.963731
death                     0.957738
other_aid                 0.865990
infrastructure_related    0.934839
transport                 0.954511
buildings                 0.949900
electricity               0.980636
tools                     0.994467
hospitals                 0.987706
shops                     0.994621
aid_centers               0.988013
other_infrastructure      0.957277
weather_related           0.866452
floods    

In [15]:
print(classification_report(y_test.values, y_pred, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.97      0.89      5011
               request       0.92      0.43      0.58      1099
                 offer       0.00      0.00      0.00        36
           aid_related       0.79      0.62      0.70      2741
          medical_help       0.74      0.05      0.10       546
      medical_products       0.71      0.07      0.13       332
     search_and_rescue       0.61      0.06      0.11       188
              security       0.00      0.00      0.00       111
              military       0.67      0.03      0.06       207
                 water       0.88      0.34      0.49       425
                  food       0.86      0.39      0.54       730
               shelter       0.90      0.21      0.35       606
              clothing       0.60      0.03      0.05       105
                 money       1.00      0.02      0.04       159
        missing_people       1.00      

In [16]:
%%time

# train classifier
xgb_pipeline.fit(X_train, y_train)

# predict on test data
y_pred_xgb = xgb_pipeline.predict(X_test)

# display results
display_results(y_test, y_pred_xgb)

Accuracy: related                   0.820501
request                   0.901798
offer                     0.994314
aid_related               0.777009
medical_help              0.922545
medical_products          0.954972
search_and_rescue         0.971569
security                  0.982480
military                  0.972030
water                     0.966805
food                      0.950515
shelter                   0.948517
clothing                  0.989089
money                     0.975718
missing_people            0.988474
refugees                  0.967881
death                     0.969418
other_aid                 0.868449
infrastructure_related    0.934378
transport                 0.957430
buildings                 0.958660
electricity               0.982634
tools                     0.994467
hospitals                 0.988167
shops                     0.994467
aid_centers               0.988474
other_infrastructure      0.955586
weather_related           0.879668
floods    

In [17]:
print(classification_report(y_test.values, y_pred_xgb, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.85      0.93      0.89      5011
               request       0.78      0.58      0.67      1099
                 offer       0.00      0.00      0.00        36
           aid_related       0.78      0.66      0.71      2741
          medical_help       0.58      0.27      0.37       546
      medical_products       0.63      0.28      0.39       332
     search_and_rescue       0.53      0.16      0.25       188
              security       0.33      0.03      0.05       111
              military       0.63      0.29      0.40       207
                 water       0.79      0.66      0.72       425
                  food       0.80      0.75      0.77       730
               shelter       0.79      0.60      0.69       606
              clothing       0.79      0.44      0.56       105
                 money       0.51      0.23      0.31       159
        missing_people       0.57      

/home/ajana/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ajana/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/ajana/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


From the results, two things can be inferred, there is something wrong with **related** column and **child_alone** column.

In [11]:
# investigate "related" and "child-alone" column
Y["related"].value_counts()

1    19906
0     6122
Name: related, dtype: int64

In [12]:
Y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

Few importatnt words like **water, blocked road, medical supplies** are used during a disaster response. So we can create a Custom Transformers like **StartingNounExtractor**, **StartingVerbExtractor** and, **LengthExtractor** and add them to our pipeline.


In [8]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)


In [9]:
# https://www.guru99.com/pos-tagging-chunking-nltk.html

class StartingNounExtractor(BaseEstimator, TransformerMixin):

    def starting_noun(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['NN', 'NNS', 'NNP', 'NNPS'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_noun)
        return pd.DataFrame(X_tagged)

In [25]:
# Not useful in this case
class LengthExtractor(BaseEstimator, TransformerMixin): 
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return  pd.Series(X).apply(len).values.reshape(-1,1)

### Using FeatureUnion

In [31]:
rand_pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            #('length', LengthExtractor()),
            #('starting_noun', StartingNounExtractor()),
            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 100)))
    ])

In [10]:

boost_pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            #('length', LengthExtractor()),
            ('starting_noun', StartingNounExtractor()),
            ('starting_verb', StartingVerbExtractor())
        ])),

        ('xgbclassifier', MultiOutputClassifier(XGBClassifier(objective='binary:logistic',random_state = 42)))
    ])

In [32]:
%%time

# train classifier
rand_pipeline.fit(X_train, y_train)

#predict on test data
y_pred_rand = rand_pipeline.predict(X_test)

#display results
display_results(y_test, y_pred_rand)

Accuracy: related                   0.810973
request                   0.891963
offer                     0.994467
aid_related               0.769325
medical_help              0.918549
medical_products          0.950822
search_and_rescue         0.971723
security                  0.982941
military                  0.968495
water                     0.950054
food                      0.928538
shelter                   0.926080
clothing                  0.984478
money                     0.976026
missing_people            0.988167
refugees                  0.963270
death                     0.957584
other_aid                 0.864915
infrastructure_related    0.934993
transport                 0.953435
buildings                 0.950515
electricity               0.980483
tools                     0.994467
hospitals                 0.987706
shops                     0.994621
aid_centers               0.988013
other_infrastructure      0.957277
weather_related           0.867066
floods    

In [46]:
%%time

# train classifier
boost_pipeline.fit(X_train, y_train)

#predict on test data
y_pred_boost = boost_pipeline.predict(X_test)

#display results
display_results(y_test, y_pred_boost)

Accuracy: related                   0.822499
request                   0.901491
offer                     0.994314
aid_related               0.778393
medical_help              0.923313
medical_products          0.956047
search_and_rescue         0.971569
security                  0.982019
military                  0.972491
water                     0.965729
food                      0.952666
shelter                   0.948210
clothing                  0.988781
money                     0.976641
missing_people            0.988935
refugees                  0.967573
death                     0.970954
other_aid                 0.870140
infrastructure_related    0.933764
transport                 0.956969
buildings                 0.958506
electricity               0.982327
tools                     0.994467
hospitals                 0.987859
shops                     0.994621
aid_centers               0.988474
other_infrastructure      0.954972
weather_related           0.878746
floods    

As we can see adding Custom Transformers like **StartingNounExtractor**, **StartingVerbExtractor** to our pipeline, improves the accuracy. Also, XGBoost classifier workes better than random forest. So we'll apply GridsearchCV on XGBoost.
**LengthExtractor** degrades accuracy.


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
#REF : https://xgboost.readthedocs.io/en/latest/python/python_api.html
# https://www.kaggle.com/tilii7/hyperparameter-grid-search-with-xgboost
parameters = {
#         'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#         'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
#         'features__text_pipeline__vect__max_features': (None, 5000, 10000),
#         'features__text_pipeline__tfidf__use_idf': (True, False),
#         'xgbclassifier__estimator__n_estimators': [50, 1000],
        'xgbclassifier__estimator__learning_rate': [0.1, 0.5],
#         'xgbclassifier__estimator__max_depth': [3,5],
#         'xgbclassifier__estimator__gamma': [0.5, 2, 5],
#         'features__transformer_weights': (
#             {'text_pipeline': 1, 'starting_verb': 0.5,'starting_noun': 0.5},
#             {'text_pipeline': 0.5, 'starting_verb': 1,'starting_noun': 0.5},
#             {'text_pipeline': 1, 'starting_verb': 0.5,'starting_noun': 1},
#             {'text_pipeline': 0.8, 'starting_verb': 1,'starting_noun': 0.5},
#         )
    }

cv = GridSearchCV(boost_pipeline, param_grid=parameters,cv = 5)


In [ ]:
%%time
cv.fit(X_train, y_train)

In [ ]:
# predict on test data
y_pred_final = cv.predict(X_test)

# display results
display_results(y_test, y_pred_final)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
print(classification_report(y_test.values, y_pred_final, target_names=Y.columns.values))

### 9. Export your model as a pickle file

In [ ]:
import pickle

pickle.dump(cv, open('models/classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.